## Customer Support Chatbot: AI-Powered Librarian



In [ ]:
! pip install openai==0.28 comet-llm --quiet

In [ ]:
# libraries
import openai
import os
import IPython
import comet_llm
from comet_llm import Span, end_chain, start_chain

# API configuration
openai.api_key = "OPENAI_API_KEY"

COMET_API_KEY = "COMET_API_KEY"
COMET_WORKSPACE = "COMET_WORKSPACE"

comet_llm.init(project="book-chatbot-evaluation", api_key=COMET_API_KEY)

Here is a helper function to load the results from the OpenAI APIs:

In [ ]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=300):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

The following is a list of books which will be added as context in the prompt design in the next step:

In [ ]:
books = """
Category: Children's Books
Book Title: The Little Blue Engine
Price: $6.99
Illustrated: Y
Popularity: 4/5
Summary: A brave little train takes on a challenging journey, teaching children about perseverance and self-belief.

Category: Mystery Novels
Book Title: Shadows Over Elm Street
Price: $8.99
Illustrated: N
Popularity: 3/5
Summary: A detective follows mysterious clues leading to a dark secret buried in a small town.

Category: Mystery Novels
Book Title: The Venetian Masquerade
Price: $7.99
Illustrated: N
Popularity: 4/5
Summary: In the heart of Venice, a masked ball turns to mayhem, and one detective must unmask the truth.

Category: Romance Novels
Book Title: Moonlit Whispers
Price: $12.99
Illustrated: N
Popularity: 4/5
Summary: Two strangers, one chance meeting, and a romance that transcends lifetimes.

Category: Romance Novels
Book Title: The Cafe by the Sea
Price: $10.99
Illustrated: N
Popularity: 5/5
Summary: A city girl finds love and rediscovery in a small coastal town, along with the charms of local life.

Category: Romance Novels
Book Title: Heartstrings in Tuscany
Price: $14.99
Illustrated: N
Popularity: 4/5
Summary: An unexpected journey to Italy leads to a summer of romance, secrets, and vineyard nights.

Category: Science Fiction
Book Title: Nebula's Edge
Price: $9.99
Illustrated: N
Popularity: 3/5
Summary: In a galaxy on the brink of war, one spaceship captain must navigate politics, love, and space pirates.

Category: Science Fiction
Book Title: Beyond Earth's Echo
Price: $11.99
Illustrated: Y
Popularity: 4/5
Summary: Humanity ventures to a new planet, only to find traces of a long-lost alien civilization.

Category: Poetry Collections
Book Title: Echoes of Midnight
Price: $6.99
Illustrated: N
Popularity: 5/5
Summary: A captivating collection of poems about love, loss, and the nuances of the human soul.

Category: Poetry Collections
Book Title: Sunflowers & Daydreams
Price: $5.99
Illustrated: Y
Popularity: 4/5
Summary: Vibrant verses celebrating nature, life's simple joys, and moments of quiet reflection.
"""

There are questions which we will use to test the chatbot:

In [ ]:
questions = [
    "Do you have science fiction books?",
    "Do you have any illustrated book options?",
    "How many different kinds of science fiction books do you have?",
    "What's the price for the Harry Potter book?",
    "What are the two most popular books?",
    "Do you have any books under $7?",
    "Does any of your science fiction books contain alien characters?"
]

The code below both retrieves an entire response (via chain-of-thought) and then extracts the final response based on the question asked by the user. For every question, it logs the results from the CoT and the final response using Comet's prompt chains logging functionalities. Please reference the corresponding video lecture for the full code explanation.

In [ ]:
chatbot_responses = []

# return the final response to the user
def get_only_response(response):
  messages = [
    {
      "role": "system",
      "content": "Your task is to extract only the response to the user in the following full chatbot response: {response}".format(response=response)
    }
  ]

  return get_completion(messages)

# loop through the questions and try to answer them using a chain-of-thought prompt
for question in questions:
  messages=[
    {
      "role": "system",
      "content": "Your task is to answer questions factually about a book catalogue, provided below and delimited by +++++. The user request is provided here: {request}\n\nStep 1: The first step is to check if the user is asking a question related to any type of book (even if that book item is not on the list). If the question is about any type of food, we move on to Step 2 and ignore the rest of Step 1. If the question is not about book, then you send a response: \"Sorry! I cannot help with that. Please let me know if you have a question about our book catalogue.\"\n\nStep 2: In this step, you check that the user question is relevant to any of the items on the book catalogue. You should check that the book item exists in the book catalogue. If it doesn't exist then send a kind response to the user that the item doesn't exist in the catalogue and then include a list of available but similar book items without any other details (e.g., price). The book items available are provided below and delimited by +++++: {book_titles}+++++\n\nStep 3: If the item exists in the book catalogue and the user is requesting specific information, provide that relevant information to the user using the book catalogue. Make sure to use a friendly tone and keep the response concise.\n\nPerform the following reasoning steps to send a response to the user:\nStep 1: <Step 1 reasoning>\nStep 2: <Step 2 reasoning>\nResponse to the user (only output the final response): <response to user>".format(request=question, book_titles=books)
    }
  ]

  # get the full response
  response = get_completion(messages)

  # save responses
  chatbot_responses.append(response)

  # start chain
  start_chain(
    inputs={"question": question},
    api_key=COMET_API_KEY,
  )

  # store full response
  with Span(
    category="reasoning",
    name="chain-of-thought",
    inputs={"user_question": question},
    ) as span:
      span.set_outputs(outputs={"full_response": response})

  # end chain with the end response after processing it
  with Span(
    category="response-extraction",
    inputs={
        "user_question": question,
        "full_response": response,
    },
  ) as span:
    # process response to take out only the response to the user
    final_response = get_only_response(response)

    span.set_outputs(outputs={"final_response": final_response})

  end_chain(outputs={"final_response": final_response})


Chain logged to https://www.comet.com/omarsar/book-chatbot-evaluation
Step 1: The user is asking a question related to a type of book.

Step 2: The user is asking if there are science fiction books available.

Response to the user: Yes, we have science fiction books available. Here are the science fiction books in our catalogue:

1. Book Title: Nebula's Edge
   Price: $9.99
   Illustrated: No
   Popularity: 3/5
   Summary: In a galaxy on the brink of war, one spaceship captain must navigate politics, love, and space pirates.

2. Book Title: Beyond Earth's Echo
   Price: $11.99
   Illustrated: Yes
   Popularity: 4/5
   Summary: Humanity ventures to a new planet, only to find traces of a long-lost alien civilization.
